# Getting Date column for shot data

### handmade testing (ignore/delete if logic makes sense)

In [4]:
import requests
import pandas as pd
shots = pd.read_csv('/Users/dB/Documents/repos/github/hockey-clutch2-model/data/shot_data/combined_shots_no_dates.csv')

In [7]:
sample_seasons = shots['season'].unique()
sample_seasons

array([2023., 2022., 2021., 2020., 2019., 2018., 2007., 2008., 2009.,
       2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017., 2024.])

In [6]:
shots.head()

,shotID,homeTeamCode,awayTeamCode,season,isPlayoffGame,game_id,homeTeamWon,id,time,timeUntilNextEvent,...,xFroze,xRebound,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xShotWasOnGoal,isHomeTeam,shotWasOnGoal,teamCode,arenaAdjustedXCordABS
0,0.0,TBL,NSH,2023.0,0.0,20001.0,1.0,10.0,61.0,9.0,...,0.249174,0.035816,0.379310,0.277143,0.022744,0.751320,1.0,1.0,TBL,59.0
1,1.0,TBL,NSH,2023.0,0.0,20001.0,1.0,11.0,70.0,26.0,...,0.107945,0.051284,0.464358,0.298119,0.020588,0.589712,1.0,1.0,TBL,81.0
2,2.0,TBL,NSH,2023.0,0.0,20001.0,1.0,13.0,107.0,2.0,...,0.236193,0.030269,0.390417,0.301375,0.025620,0.785765,1.0,1.0,TBL,55.0
3,3.0,TBL,NSH,2023.0,0.0,20001.0,1.0,19.0,177.0,35.0,...,0.198635,0.033686,0.438052,0.291724,0.021107,0.724471,1.0,1.0,TBL,58.0
4,4.0,TBL,NSH,2023.0,0.0,20001.0,1.0,21.0,231.0,29.0,...,0.188582,0.027146,0.406288,0.349752,0.019523,0.727481,0.0,0.0,NSH,64.0


In [8]:
sample_games = shots['game_id'].unique()
df = pd.DataFrame({"season":sample_seasons, "game_id":sample_games}, index=list(range(len(sample_games))))

ValueError: Length of values (18) does not match length of index (1461)

In [79]:
def create_composite_game_id(season, game_id):
  #this should keep the 6 digit game id padding and prefix accordingly
  if game_id < 100000:
    return f"{season}0{game_id:05d}"
  else:
    return f"{season}{game_id:06d}"

In [62]:
# each new column can have one of 3 values:
# correct string, NA, or 'NoX' (if parsing failed/response doest have date; neither of which should happen)
def use_game_endpoint(composite_game_id):
  endpoint = "https://api-web.nhle.com/v1/meta/game/"+ composite_game_id #c_g_id column should be a string
  response = requests.get(endpoint)
  if response.status_code == 200:
    date = response.json()['seasonStates'].get('date','NoDate')
    team1 = response.json()['teams'][0].get('tricode', 'NoTeam1')
    team2 = response.json()['teams'][1].get('tricode', 'NoTeam2')
    return date, team1, team2

  else:
    return pd.NA, pd.NA, pd.NA

In [69]:
df2 = df.copy()
df2['composite_game_id'] = df2.apply(lambda row: create_composite_game_id(row['season'], row['game_id']), axis=1)
df2[['game_date', 'team1', 'team2']] = df2['composite_game_id'].apply(lambda x: pd.Series(use_game_endpoint(x)))
df2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   season             6 non-null      int64 
 1   game_id            6 non-null      int64 
 2   composite_game_id  6 non-null      object
 3   game_date          6 non-null      object
 4   team1              6 non-null      object
 5   team2              6 non-null      object
dtypes: int64(2), object(4)
memory usage: 508.0+ bytes


In [77]:
# dont know if this will end up being needed but technically theres no need to keep using the endpoint for each row
from collections import defaultdict
cache = defaultdict()
def use_game_endpoint_w_cache(composite_game_id):
  if composite_game_id in cache:
    print(f'{composite_game_id} already existed in cache')
    return cache[composite_game_id]


  endpoint = "https://api-web.nhle.com/v1/meta/game/"+ composite_game_id #c_g_id column should be a string
  print(f'looked up {composite_game_id}')
  response = requests.get(endpoint)
  if response.status_code == 200:
    date = response.json()['seasonStates'].get('date','NoDate')
    cache[composite_game_id] = date
    return cache[composite_game_id]

  else:
    cache[composite_game_id] = pd.NA
    return cache[composite_game_id]

In [78]:
df3 = df.copy()
df3['composite_game_id'] = df3.apply(lambda row: create_composite_game_id(row['season'], row['game_id']), axis=1)
df3['game_date'] = df3.apply(lambda row: use_game_endpoint_w_cache(row['composite_game_id']), axis=1)
df3


looked up 2011020001
looked up 2011020002
looked up 2023020001
2023020001 already existed in cache
looked up 2024020001
2024020001 already existed in cache


,season,game_id,composite_game_id,game_date
0,2011,20001,2011020001,2011-10-06
1,2011,20002,2011020002,2011-10-06
2,2023,20001,2023020001,2023-10-10
3,2023,20001,2023020001,2023-10-10
4,2024,20001,2024020001,2024-10-04
5,2024,20001,2024020001,2024-10-04


In [68]:
df2
# 2011 Season
# https://www.hockey-reference.com/leagues/NHL_2012_games.html
# 2023 Season
# https://www.hockey-reference.com/leagues/NHL_2024_games.html
# 2024 Season
# https://www.hockey-reference.com/leagues/NHL_2025_games.html

# For basic sanity check --> Looks good

,season,game_id,composite_game_id,game_date,team1,team2
0,2011,20001,2011020001,2011-10-06,BOS,PHI
1,2011,20002,2011020002,2011-10-06,MTL,TOR
2,2023,20001,2023020001,2023-10-10,NSH,TBL
3,2023,20001,2023020001,2023-10-10,NSH,TBL
4,2024,20001,2024020001,2024-10-04,BUF,NJD
5,2024,20001,2024020001,2024-10-04,BUF,NJD


## GPT Refined

In [80]:
import requests
import pandas as pd
from collections import defaultdict

# Cache to store fetched results
cache = defaultdict()

In [85]:
def create_game_cols(season, game_id):
    """
    Creates a composite_game_id, fetches game_date using an API, and caches results.

    Parameters:
    - season (int): The season year.
    - game_id (int): The game ID.

    Returns:
    - composite_game_id (str): The formatted composite_game_id.
    - game_date (str or pd.NA): The date of the game or NaN if not found.
    """
    # Step 1: Create composite_game_id
    if game_id < 100000:
        composite_game_id = f"{season}0{game_id:05d}"
    else:
        composite_game_id = f"{season}{game_id:06d}"

    # Step 2: Check cache
    if composite_game_id in cache:
        #print(f'DEBUG: already existed in cache {composite_game_id} ')
        return composite_game_id, cache[composite_game_id]

    # Step 3: Fetch from API
    #print(f'DEBUG: looked up {composite_game_id}')
    endpoint = f"https://api-web.nhle.com/v1/meta/game/{composite_game_id}"
    response = requests.get(endpoint)

    if response.status_code == 200:
        game_date = response.json().get('seasonStates', {}).get('date', 'NoDate')
        cache[composite_game_id] = game_date
        return composite_game_id, game_date
    else:
        cache[composite_game_id] = pd.NA
        return composite_game_id, pd.NA

In [86]:
df[['composite_game_id', 'game_date']] = df.apply(lambda row: pd.Series(create_game_cols(row['season'], row['game_id'])), axis=1)

In [87]:
df

,season,game_id,composite_game_id,game_date
0,2011,20001,2011020001,2011-10-06
1,2011,20002,2011020002,2011-10-06
2,2023,20001,2023020001,2023-10-10
3,2023,20001,2023020001,2023-10-10
4,2024,20001,2024020001,2024-10-04
5,2024,20001,2024020001,2024-10-04


# Cleaning Shot Data

We downloaded shot data from MoneyPuck, but the current season's data was not organized in the same way as the historical data. This notebook aims to connect these two data sources using pandas.

In [ ]:
import pandas as pd

In [ ]:
hist_data = pd.read_csv('/Users/dB/Desktop/fall_24/DS-4210/final-proj/data/shots_2007-2023.csv')
curr_data = pd.read_csv('/Users/dB/Desktop/fall_24/DS-4210/final-proj/data/shots_2024.csv')

In [ ]:
hist_data.head()

,shotID,homeTeamCode,awayTeamCode,season,isPlayoffGame,game_id,homeTeamWon,id,time,timeUntilNextEvent,...,xFroze,xRebound,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xShotWasOnGoal,isHomeTeam,shotWasOnGoal,teamCode,arenaAdjustedXCordABS
0,0,TBL,NSH,2023,0,20001,1,10,61,9.0,...,0.249174,0.035816,0.379310,0.277143,0.022744,0.751320,1.0,1.0,TBL,59.0
1,1,TBL,NSH,2023,0,20001,1,11,70,26.0,...,0.107945,0.051284,0.464358,0.298119,0.020588,0.589712,1.0,1.0,TBL,81.0
2,2,TBL,NSH,2023,0,20001,1,13,107,2.0,...,0.236193,0.030269,0.390417,0.301375,0.025620,0.785765,1.0,1.0,TBL,55.0
3,3,TBL,NSH,2023,0,20001,1,19,177,35.0,...,0.198635,0.033686,0.438052,0.291724,0.021107,0.724471,1.0,1.0,TBL,58.0
4,4,TBL,NSH,2023,0,20001,1,21,231,29.0,...,0.188582,0.027146,0.406288,0.349752,0.019523,0.727481,0.0,0.0,NSH,64.0


In [ ]:
curr_data.head()

,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,...,xCordAdjusted,xFroze,xGoal,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xRebound,xShotWasOnGoal,yCord,yCordAdjusted
0,0,52.0,57.0,57.0,-41.0,41.0,0.0,0,0,0,...,57,0.238455,0.012537,0.394229,0.301072,0.022807,0.030900,0.710867,-40,-40
1,1,33.0,71.0,71.0,-28.0,28.0,-6.0,0,0,0,...,71,0.198306,0.021962,0.404919,0.313773,0.023774,0.037266,0.759039,-28,-28
2,2,48.0,48.0,48.0,-24.0,24.0,-12.6,0,0,0,...,48,0.213829,0.028057,0.405311,0.294682,0.025849,0.032272,0.696901,-24,-24
3,3,58.0,-40.0,40.0,-31.0,31.0,0.0,0,0,0,...,41,0.209478,0.009832,0.449775,0.277671,0.019667,0.033577,0.610530,-31,31
4,4,56.0,-35.0,35.0,15.0,15.0,0.0,0,0,0,...,36,0.376712,0.028884,0.307725,0.205568,0.022266,0.058845,0.799576,15,-15


## Function to compare 2 dataframes

Because there are too many columns to make sure that the columns of each dataframe match, this will be done programmatically. The following function will take 2 dataframes, compare the column names, and output if the column names match, and output any missing columns from each dataframe if any exist.

###### This code was generated with Claude 3.5 Sonnet

In [ ]:
def compare_dataframe_columns(df1, df2):
    """
    Compare two dataframes to check if they have the same columns, regardless of order.
    Returns a tuple containing:
    - Boolean indicating if columns match exactly
    - Set of columns in df1 but not in df2 (if any)
    - Set of columns in df2 but not in df1 (if any)

    Parameters:
    -----------
    df1 : pandas.DataFrame
        First DataFrame to compare
    df2 : pandas.DataFrame
        Second DataFrame to compare

    Returns:
    --------
    tuple : (bool, set, set)
        - True if columns match exactly, False otherwise
        - Columns unique to df1
        - Columns unique to df2

    Runtime Complexity: O(n) where n is the total number of columns
    Space Complexity: O(n) to store the column sets
    """
    # Convert column lists to sets for O(1) lookup time
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)

    # Find columns unique to each DataFrame
    cols_only_in_df1 = cols1 - cols2
    cols_only_in_df2 = cols2 - cols1

    # Check if the columns match exactly
    columns_match = len(cols_only_in_df1) == 0 and len(cols_only_in_df2) == 0

    return columns_match, cols_only_in_df1, cols_only_in_df2

In [ ]:
compare_dataframe_columns(hist_data, curr_data)

(False,
 set(),
 {'gameOver',
  'homeTeamScore',
  'homeWinProbability',
  'penaltyLength',
  'playoffGame',
  'roadTeamCode',
  'roadTeamScore',
  'shotGoalProbability',
  'shotPlayContinued',
  'timeBetweenEvents',
  'timeLeft',
  'wentToOT',
  'wentToShootout'})

## Appending the Current Data to the Historical Data

Because none of the missing items in the historical dataset were deemed important, the team decided to append the datasets anyways. This function appends the dataframe and removes the unnecessary columns.

In [ ]:
def append_matching_columns(df1, df2):
    """
    Append df2 to df1, keeping only columns that exist in df1.
    Maintains the column order of df1 in the result.

    Parameters:
    -----------
    df1 : pandas.DataFrame
        Base DataFrame that defines which columns to keep
    df2 : pandas.DataFrame
        DataFrame to append, matching df1's columns

    Returns:
    --------
    pandas.DataFrame
        Combined DataFrame with only df1's columns

    Runtime Complexity: O(n) where n is the number of rows in df2
    Space Complexity: O(n) for the new combined DataFrame
    """
    # Get the columns from df1 that exist in df2
    # Using list comprehension instead of sets for ordered comparison
    common_cols = [col for col in df1.columns if col in df2.columns]

    # Select only matching columns from df2 and append to df1
    # Using concat instead of append() as it's more efficient
    return pd.concat(
        [
            df1,
            df2[common_cols].reindex(columns=df1.columns)
        ],
        axis=0,
        ignore_index=True
    )

In [ ]:
combined_data = append_matching_columns(hist_data, curr_data)

In [ ]:
combined_data.head()

,shotID,homeTeamCode,awayTeamCode,season,isPlayoffGame,game_id,homeTeamWon,id,time,timeUntilNextEvent,...,xFroze,xRebound,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xShotWasOnGoal,isHomeTeam,shotWasOnGoal,teamCode,arenaAdjustedXCordABS
0,0,TBL,NSH,2023,0,20001,1,10,61,9.0,...,0.249174,0.035816,0.379310,0.277143,0.022744,0.751320,1.0,1.0,TBL,59.0
1,1,TBL,NSH,2023,0,20001,1,11,70,26.0,...,0.107945,0.051284,0.464358,0.298119,0.020588,0.589712,1.0,1.0,TBL,81.0
2,2,TBL,NSH,2023,0,20001,1,13,107,2.0,...,0.236193,0.030269,0.390417,0.301375,0.025620,0.785765,1.0,1.0,TBL,55.0
3,3,TBL,NSH,2023,0,20001,1,19,177,35.0,...,0.198635,0.033686,0.438052,0.291724,0.021107,0.724471,1.0,1.0,TBL,58.0
4,4,TBL,NSH,2023,0,20001,1,21,231,29.0,...,0.188582,0.027146,0.406288,0.349752,0.019523,0.727481,0.0,0.0,NSH,64.0


## Writing the New Historical Data to a File

In [ ]:
def filter_recent_years(df, year_column, min_year=2019):
    """
    Remove rows from DataFrame where the year is older than the specified minimum year.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame to filter
    year_column : str
        Name of the column containing year values
    min_year : int, optional
        Minimum year to keep (default: 2019)

    Returns:
    --------
    pandas.DataFrame
        Filtered DataFrame containing only rows from min_year and newer

    Raises:
    -------
    KeyError: If year_column doesn't exist in DataFrame
    ValueError: If year values can't be converted to integers

    Runtime Complexity: O(n) where n is the number of rows
    """
    # Verify column exists
    if year_column not in df.columns:
        raise KeyError(f"Year column '{year_column}' not found in DataFrame")

    try:
        # Convert year column to numeric if it isn't already
        # This handles various formats like strings, floats, etc.
        years = pd.to_numeric(df[year_column])

        # Create year mask and filter DataFrame
        # Using boolean indexing which is very efficient in pandas
        year_mask = years >= min_year

        # Return filtered DataFrame
        return df[year_mask].copy()

    except ValueError as e:
        raise ValueError(f"Unable to convert values in '{year_column}' to numeric years: {str(e)}")

In [ ]:
df_to_write = filter_recent_years(combined_data, "season", 2019)

In [ ]:
df_to_write.to_csv('/Users/dB/Desktop/fall_24/DS-4210/final-proj/data/combined_dataset.csv')